## Opening LiveOcean Forecast Data from a Kerchunk Reference File

In [9]:
import fsspec
import xarray as xr
import zstandard
import hvplot.xarray
#kerchunk is not needed to open the data only fsspec to create the mapper

In [11]:
# reference_file = 's3://esip-qhub-public/LiveOcean/LiveOcean_reference.json'  #url for where reference json has been saved 
reference_file = '/Users/pm8/Documents/LPM_output/LiveOcean_reference.json'

In [12]:
# topts = dict(anon=True, skip_instance_cache=True) # target options for opening the reference jsons
topts = dict(anon=True, skip_instance_cache=True) # target options for opening the reference jsons
ropts = dict(account_name='pm2', skip_instance_cache=True) # remote options for opening the netcdf files 
#skip instance cache is only neccassary if you believe the files have recently changed

In [13]:
# fs = fsspec.filesystem("reference", fo=reference_file, target_options=topts,
#                        remote_protocol='abfs', remote_options=ropts) #create reference file system
fs = fsspec.filesystem("reference", fo=reference_file,
                       remote_protocol='abfs', remote_options=ropts) #create reference file system
m = fs.get_mapper("") #open mapper pointing to reference file system
ds = xr.open_dataset(m, engine="zarr", consolidated=False) # open live ocean data as if it where a zarr file!
ds 

<xarray.Dataset>
Dimensions:         (ocean_time: 3, s_w: 31, eta_rho: 1302, xi_rho: 663,
                     tracer: 11, s_rho: 30, boundary: 4, eta_u: 1302,
                     xi_u: 662, eta_v: 1301, xi_v: 663, eta_psi: 1301,
                     xi_psi: 662)
Coordinates:
    lat_psi         (eta_psi, xi_psi) float64 ...
    lat_rho         (eta_rho, xi_rho) float64 ...
    lat_u           (eta_u, xi_u) float64 ...
    lat_v           (eta_v, xi_v) float64 ...
    lon_psi         (eta_psi, xi_psi) float64 ...
    lon_rho         (eta_rho, xi_rho) float64 ...
    lon_u           (eta_u, xi_u) float64 ...
    lon_v           (eta_v, xi_v) float64 ...
  * ocean_time      (ocean_time) datetime64[ns] 2022-08-14T22:00:00 ... 2022-...
  * s_rho           (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w             (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.06667 -0.03333 0.0
Dimensions without coordinates: eta_rho, xi_rho, tracer, boundary, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/138)
    AKs             (ocean_time, s_w, eta_rho, xi_rho) float32 ...
    AKv             (ocean_time, s_w, eta_rho, xi_rho) float32 ...
    Akk_bak         (ocean_time) float64 ...
    Akp_bak         (ocean_time) float64 ...
    Akt_bak         (ocean_time, tracer) float64 ...
    Akv_bak         (ocean_time) float64 ...
    ...              ...
    zooFegest       (ocean_time) float64 ...
    zooI0           (ocean_time) float64 ...
    zooKs           (ocean_time) float64 ...
    zooMin          (ocean_time) float64 ...
    zooZeta         (ocean_time) float64 ...
    zooplankton     (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
Attributes: (12/40)
    CPP_options:       U0KB, ADD_FSOBC, ADD_M2OBC, ANA_BPFLUX, ANA_BSFLUX, AN...
    Conventions:       CF-1.4, SGRID-0.3
    NLM_LBC:           \nEDGE:           WEST   SOUTH  EAST   NORTH  \nzeta: ...
    ana_file:          ROMS/Functionals/ana_btflux.h, ROMS/Functionals/ana_st...
    bio_file:          ROMS/Nonlinear/Biology/npzd2o_banas.h
    bpar_file:         /gscratch/macc/parker/LO_roms/cas6_v0_u0kb/f2022.08.14...
    ...                ...
    svn_rev:           824M
    svn_url:           https://www.myroms.org/svn/src/trunk
    tiling:            020x020
    title:             First LiveOcean input file
    type:              ROMS/TOMS history file
    var_info:          /gscratch/macc/parker/LiveOcean_roms/LO_ROMS/ROMS/Exte...

In [14]:
%%time
#plot timeseries of ocean salinity at the surface
#this computation was done on an AWS instance in us-west2 and should be even quicker closer to the source data
surface_salinity = ds.salt.sel(s_rho = 0, method = 'nearest')
surface_salinity[:,10,10].hvplot(x = 'ocean_time')

CPU times: user 118 ms, sys: 48.7 ms, total: 166 ms
Wall time: 1.08 s


:Curve   [ocean_time]   (salt)

In [15]:
%%time 
#select a specific date and level to plot
ocean_temperature = ds.temp[-1,-1,:,:].load() #loading into memory is not necessary but will greatly speed up the map load time
ocean_temperature

CPU times: user 242 ms, sys: 92.6 ms, total: 335 ms
Wall time: 1.77 s


<xarray.DataArray 'temp' (eta_rho: 1302, xi_rho: 663)>
array([[18.153908, 18.167871, 18.169632, ...,       nan,       nan,
              nan],
       [18.139948, 18.177217, 18.171705, ...,       nan,       nan,
              nan],
       [18.131193, 18.18593 , 18.18196 , ...,       nan,       nan,
              nan],
       ...,
       [15.493612, 15.408571, 15.360038, ...,       nan,       nan,
              nan],
       [15.528646, 15.465902, 15.414656, ...,       nan,       nan,
              nan],
       [15.567387, 15.606144, 15.606324, ...,       nan,       nan,
              nan]], dtype=float32)
Coordinates:
    lat_rho     (eta_rho, xi_rho) float64 42.01 42.01 42.01 ... 52.01 52.01
    lon_rho     (eta_rho, xi_rho) float64 -130.0 -129.9 -129.9 ... -122.0 -122.0
    ocean_time  datetime64[ns] 2022-08-15
    s_rho       float64 -0.01667
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    field:      temperature, scalar, series
    grid:       grid
    location:   face
    long_name:  potential temperature
    time:       ocean_time
    units:      Celsius

In [16]:
#render as tile plot using holoviews
ocean_temperature.unify_chunks().hvplot.quadmesh(x='lon_rho', y='lat_rho', geo=True, 
                                                frame_width=800, frame_height=500,cmap='turbo', tiles='OSM',
                                                rasterize=True, widget_location='bottom')

HoloViews(DynamicMap, widget_location='bottom')